In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import os

In [ ]:

file_lst = glob("raw/*.csv")


In [ ]:
class RES:
    def __init__(self, features=[]):
        self.ft_n = len(features)
        self.features=features
        self.RES = np.empty((1, self.ft_n))
        self.RES_len=0
        self.RES_cap=1

    def add(self,x):
        if isinstance(x, pd.DataFrame):
            x = x.to_numpy()        
        size = x.shape[0]
        if (size + self.RES_len > self.RES_cap):
            while (size + self.RES_len > self.RES_cap):
                self.RES_cap *= 2
            
            newRes = np.empty((self.RES_cap, self.ft_n))
            newRes[0:self.RES_len,:] = self.RES[0:self.RES_len,:]
            self.RES = newRes
        self.RES[self.RES_len:self.RES_len+size, :] = x
        self.RES_len += size

    def getDf(self):
        return pd.DataFrame(self.RES[0:self.RES_len,:], columns=self.features)

    def __len__(self):
        return self.RES_len
        
    def __str__(self):
        return self.RES.__str__()


In [ ]:

n_normal = 0
n_attack = 0
req_ft=['duration','src_bytes','dst_bytes','missed_bytes','src_pkts','dst_pkts','label']
print(len(req_ft))
r = RES(req_ft)
for f in file_lst:    
    df = pd.read_csv(f, low_memory=False)  
    df = df[df['type'].isin(['normal', 'dos', 'ddos'])]
    df = df.loc[:, df.columns.isin(req_ft)]
    size = df.shape[0]
    
    at_df = df[df['label'] == 1]
    nr_df = df[df['label'] == 0]
    n_nr = nr_df.shape[0]
    n_normal += n_nr
    n_attack += at_df.shape[0]    

    print(len(r))    
    
    rng = np.random.default_rng()
    index=np.sort(rng.choice([i for i in range(size)], size=int(size*0.1), replace=False, shuffle=False))
    r.add(df.to_numpy()[index])
    del df
    


In [ ]:
print (f"NORMAL: {n_normal} ATTACK: {n_attack}")

In [ ]:
df = r.getDf()
if not os.path.exists('processed'):
        os.makedirs('processed')    
    
df.to_csv("processed/train_data.csv", index=False)

In [ ]:
df.tail(15)